# OPF

In [80]:
import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220., name="Bus1")
bus2 = pp.create_bus(net, vn_kv=110., name="Bus2")
bus3 = pp.create_bus(net, vn_kv=110., name="Bus3")
bus4 = pp.create_bus(net, vn_kv=110., name="Bus4")

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

In [81]:
# Loss Minimization

costeg = pp.create_poly_cost(net, element=eg, et='ext_grid', cp1_eur_per_mw=10)
costgen0 = pp.create_poly_cost(net, element=g0, et='gen', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, element=g1, et='gen', cp1_eur_per_mw=10)

In [82]:
pp.runopp(net, delta=1e-16)

In [83]:
net.res_ext_grid

,p_mw,q_mvar
0,56.530584,1.974564


In [84]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,71.309255,-1.969681,-3.713031,1.000008
1,12.303443,-1.451180,-3.712735,1.000010


In [85]:
# Individual Genrrator Costs

net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen0] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 12


In [86]:
pp.runopp(net, delta=1e-16)

In [87]:
net.res_ext_grid

,p_mw,q_mvar
0,144.559166,9.193021


In [88]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000079,8.601766,-16.426835,0.967619
1,0.000225,10.594623,-13.481007,0.989756


In [89]:
# Trafo Constraint

net.res_trafo.loading_percent

0    144.851179
Name: loading_percent, dtype: float64

In [90]:
net.trafo["max_loading_percent"] = 50

In [91]:
pp.runopp(net, delta=1e-16)

In [92]:
net.res_trafo.loading_percent

0    49.999136
Name: loading_percent, dtype: float64

In [93]:
net.res_ext_grid

,p_mw,q_mvar
0,49.953012,-2.147126


In [94]:
# Line Loading Contraints

net.res_line.loading_percent

0    22.192863
1    57.476322
2    33.473473
Name: loading_percent, dtype: float64

In [95]:
net.line["max_loading_percent"] = 50
pp.runopp(net, delta = 1e-16)


In [96]:
net.res_line.loading_percent

0    18.905589
1    49.999986
2    30.435895
Name: loading_percent, dtype: float64

In [97]:
# Voltage Constraints

net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-49.787584,4.603451,10.000000,-1.410210e-21
1,1.006024,-3.408832,60.000000,0.000000,13.095255,-5.409162e-02
2,0.993015,-5.815440,60.863554,-2.430957,14.999985,4.490944e-22
3,1.028887,-1.511326,-73.592614,-4.853496,12.000007,1.781411e-21


In [98]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net, delta=1e-16)

In [99]:
pp.runopp(net, delta = 1e-16)

# All the code above this point is part of 'OPF Tutorial'. The code for task is starting from below

# 

# LMP Function

In [105]:
# Function for Locational Marginal Prices

pp.runopp(net, delta=1e-16)

def LMP():
    
    lc = []     #list containing load changes on existing buses
    
    # A for loop to ask for load changes (if applicable) on all the existing buses.
    for load in range(len(net.bus)):
        lc.append(int(input(f"Enter the load change for bus {load}: "))) 
    print(lc)
    
    #Counter for all the buses in the n/w
    for bus in range(len(net.bus)):     
        
        g = []      #List to store MW values prior to load change. The list is initialized inside the loop so that it doesn't contain previous MW ratings every time loop is started.
        G = []      #List to store MW values post load change
        
        g.append(net.res_ext_grid.p_mw.at[0])
        g.append(net.res_gen.p_mw.at[0])
        g.append(net.res_gen.p_mw.at[1])

        #Creates a load on the current bus IF the list has a respective load value.
        if lc[bus] != 0:        
            pp.create_load(net, net.bus.index[bus], p_mw = lc[bus], controllable=False)
        
        pp.runopp(net, delta=1e-16)
        
        G.append(net.res_ext_grid.p_mw.at[0])
        G.append(net.res_gen.p_mw.at[0])
        G.append(net.res_gen.p_mw.at[1])

        #List to store the difference in generation before and after load addition
        delta_gen = []      
        for a in range(len(g)):
            delta_gen.append(G[a] - g[a])
        
        #Creating a list to store cost for each distinct power source
        cost = []
        for a in range(len(net.poly_cost.cp1_eur_per_mw)):      
            cost.append(net.poly_cost.cp1_eur_per_mw.at[a])

        lmp_cost = 0
        for k in range(len(cost)):
            lmp_cost += (delta_gen[k]*cost[k])

        print(f"The LMP at bus{bus+1} is {lmp_cost}")


In [108]:
pp.runopp(net, delta=1e-16)
LMP()

[1, 1, 1, 1]
The LMP at bus1 is 10.000004499473265
The LMP at bus2 is 13.146314471337597
The LMP at bus3 is 15.000617885675336
The LMP at bus4 is 12.000276998648896


# The Codes down below are part of my rough work. Kindly ignore them :D

In [101]:
for j in range(len(net.bus)):
    print(j)

0
1
2
3


In [76]:
cost = []
for a in range(len(net.poly_cost.cp1_eur_per_mw)):
    cost.append(net.poly_cost.cp1_eur_per_mw.at[a])
print(cost)

[10.0, 15.0, 12.0]


In [102]:
len(net.bus)

4

In [103]:
net.res_ext_grid.p_mw.at[0]

49.90683880129212

In [104]:
ab = []
for a in range(len(net.bus)):
    ab.append(net.res_ext_grid.p_mw.at[0])
print(ab)

[49.90683880129212, 49.90683880129212, 49.90683880129212, 49.90683880129212]


In [74]:
len(net.poly_cost.cp1_eur_per_mw)

3

In [78]:
kc = []
kc.append(1)
kc.append(2)
print(len(kc))

2


In [79]:
a = [1,2,3]
b = [4,5,6]
c = []
for k in range(len(a)):
    c.append(b[k] - a[k])
print(c)

[3, 3, 3]
